In [1]:
import ast
import syft as sy
from syft.core.node.common.action.function_or_constructor_action import (
    RunFunctionOrConstructorAction,
)
from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.node.common.action.run_class_method_action import RunClassMethodAction
from syft.core.pointer.pointer import Pointer

In [2]:
code_str = """
## USER CODE

{custom_code}

## Sandboxing Code
import syft
module_type = type(syft)

if "sandbox" not in syft.__dict__:
    syft.__dict__["sandbox"] = module_type(name="sandbox")

parent = syft.__dict__["sandbox"]

namespace = "mylib"

import_path = f"{namespace}.Test"
module_parts = import_path.split(".")

klass = module_parts.pop()
Test.__module__ = f"syft.sandbox.{namespace}"
Test.__name__ = klass

for n in module_parts:
    if n not in parent.__dict__:
        parent.__dict__[n] = module_type(name=n)
    parent = parent.__dict__[n]

parent.__dict__[Test.__name__] = Test

print("done", syft.sandbox.mylib)
"""

In [3]:
def send_custom_code(client, code):
    code = code_str.replace("{custom_code}", code)
    tree = ast.parse(code)
    client.secure_exec(tree)

In [4]:
def init_class(client, path: str):
    action = RunFunctionOrConstructorAction(
        path="syft.sandbox.mylib.Test",
        args=tuple(),
        kwargs={},
        id_at_location=UID(),
        address=client.address,
        msg_id=UID(),
    )

    client.send_immediate_msg_without_reply(msg=action)

    module_parts = path.split(".")
    klass_name = module_parts.pop()
    # syft.proxy.torch.TensorPointer
    fake_pointer = Pointer(id_at_location=action.id_at_location, client=client)
    fake_pointer.path_and_name=action.path
    fake_pointer.__class__.__module__ = f"syft.proxy.{'.'.join(module_parts)}"
    fake_pointer.__class__.__name__ = f"{klass_name}Pointer"
    fake_pointer.__class__.__qualname__ = f"{klass_name}Pointer"
    return fake_pointer

In [5]:
def run_on(pointer, method: str, args = [], kwargs = {}):
    action = RunClassMethodAction(
        path=f"{pointer.path_and_name}.{method}",
        _self=pointer,
        args=args,
        kwargs=kwargs,
        id_at_location=UID(),
        address=pointer.client.address,
        msg_id=UID(),
    )
    pointer.client.send_immediate_msg_without_reply(msg=action)

    fake_pointer = Pointer(id_at_location=action.id_at_location, client=pointer.client)
    fake_pointer.path_and_name = action.path
#     fake_pointer.__class__.__module__ = f"syft.proxy.{'.'.join(module_parts)}"
#     fake_pointer.__class__.__name__ = f"{klass_name}Pointer"
#     fake_pointer.__class__.__qualname__ = f"{klass_name}Pointer"
    return fake_pointer

In [6]:
# use new syckle secure custom code delivery and remote execution

In [7]:
domain = sy.Domain(name="Bob")
root_client = domain.get_root_client()

In [8]:
# first we must create the remote code

our_code = """
class Test:
    def hello(self) -> str:
        return "syckle hello"
"""

send_custom_code(root_client, our_code)

Remote Secure Exec
done <module 'mylib'>
Accepting, code executed successfully!


In [9]:
# now we just use these custom functions to execute without knowledge of the AST
test_ptr = init_class(root_client, "syft.sandbox.mylib.Test")
hello_ptr = run_on(test_ptr, "hello")

In [10]:
hello_ptr.get(delete_obj=False)

'syckle hello'

In [11]:
root_client.store.pandas

,ID,Tags,Description,object_type
0,<UID: 35cbaa0dc30b4d4d95cf830649d1d3b4>,[],,<class 'syft.sandbox.mylib.Test'>
1,<UID: 5face671061f4c7eb2195a317eb5d28c>,[],,<class 'syft.lib.python.String'>


In [12]:
import sys
sys.modules["syft"].sandbox
sy.sandbox.mylib.Test

syft.sandbox.mylib.Test